# How to Build an AI Agent from Scratch in Raw Python

Self-link: https://colab.research.google.com/drive/1g31aYu3ylEurdQjAJtnSxGFcx9aSZa_D

Tutorial by Shrinivasan Sankar: https://www.ai-bites.net/lets-build-an-ai-agent-from-scratch-in-raw-python/?ref=ai-bites-newsletter

GitHub scource code: https://github.com/ai-bites/langgraph-crash-course/blob/main/0_simple_python_agent.ipynb?ref=ai-bites.net

Video: https://youtu.be/_TzW6F1NVsc

## Tasks

Run the Colab. Understand the main idea. Understand the source code.

**Task-1:** Try to recognize the language and **automatically switching** to Chinese, English, or Russian. For example, you can ask the chatbot to automatically detect the language of the message in advance: Chinese, English or Russian. And then switch between URL types.

**Task-2 (hard):** **Weather forecast.** Add functionality to the chatbot to answer questions about the upcoming weather.

*Optional task:*  Try similar API for Chinese chatbot [QwenAI](https://chat.qwen.ai). Use [QwenAI Python API library](https://pypi.org/project/qwenai/) for tests.

*Optional task:* Try similar API for Chinese chatbot [DeepSeek](https://www.deepseek.com/). You can use the following links: [Your First API Call](https://api-docs.deepseek.com/), [An In-Depth Step-by-Step Guide](https://dev.to/auden/how-to-call-the-deepseek-r1-api-using-python-an-in-depth-step-by-step-guide-311o), [langchain_deepseek](https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html) library.

*Optional task:* Conduct a comparative analysis of the API of three chatbots: DeepSeek, OpenAI, and Qwen. Describe strengths and weaknesses of each of them. Which one do you personally find more convenient to work with?

## Install necessary libraries

In [ ]:
!pip install openai httpx dotenv

## Import necessary packages

In [ ]:
from openai import OpenAI
import re
import httpx
import requests
import xml.etree.ElementTree as ET
import json

# Print the full version information string.
import sys
print(sys.version)

# Show the Python version.
import platform
print(f"Python version: {platform.python_version()}")

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Python version: 3.11.13


**NOTE:** You have to set your OPENAI_API_KEY in a `my_openai_key.env` file and load it on Google Colab.

In [ ]:
import os
import openai

from dotenv import load_dotenv


MODEL = "gpt-4o-mini"  # Chat-GPT model name

# Set your OPENAI_API_KEY in a "my_openai_key.env" file and load it.
load_dotenv("my_openai_key_2.env")  # OPENAI_API_KEY
# !echo $OPENAI_API_KEY

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'],)

# Check to see if the LLM works.
chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Hello there!"}],
)

# The anser should be: "Hello! How can I assist you today?"
print(chat_completion.choices[0].message.content)

Hello! How can I assist you today?


## Chatbot
Create a chatbot that takes raw messages, stacks them together, and uses the LLM to respond to the messages.

In [ ]:
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})


    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.run_llm()
        self.messages.append({"role": "assistant", "content": result})
        return result


    def run_llm(self):
        completion = client.chat.completions.create(
            model=MODEL,
            temperature=0,
            messages=self.messages,
        )
        return completion.choices[0].message.content

## Agent
Wrap the above chatbot in Agent class to create ReAct Agent.

In [ ]:
# abot = Agent(prompt)
action_re = re.compile('^Action: (\w+): (.*)$')


class Agent:
    def __init__(self, system_prompt="", max_turns=1, known_actions=None):
        self.max_turns = max_turns
        self.bot = ChatBot(system_prompt)
        self.known_actions = known_actions


    def run(self, question):
        i = 0  # iteration
        next_prompt = question

        while i < self.max_turns:  # loop through iterations
            i += 1
            result = self.bot(next_prompt)
            print(f"\n Iteration: {i} \n\n"
                  f"{result}\n\n")
            actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

            if actions:  # there is an action to run
                action, action_input = actions[0].groups()
                if action not in self.known_actions:
                    raise Exception(f"Unknown action: {action}: {action_input}")
                print(f" -- running {action} {action_input} --")

                # Make a request to Wikipedia, ArXiv or calculator using action functions.
                # print(self.known_actions)
                observation = self.known_actions[action](action_input)

                next_prompt = f"Observation: {observation}"
                print("Next prompt:", next_prompt)
            else:
                return

## Actions
Define actions as functions.

The arxiv_search function will invoke arxiv API to fetch data for the agent.

The Wikipedia function will use httpx to query Wikipedia and fetch knowledge about any topic unknown to the agent.

The calculate function is a Python hack to evaluate any equations
given to it.

In [ ]:
def wikipedia(q):
    try:
        return httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": q,
            "format": "json"
        }).json()["query"]["search"][0]["snippet"]

    except IndexError:  # try Russian language instead
        return httpx.get("https://ru.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": q,
            "format": "json"
        }).json()["query"]["search"][0]["snippet"]


def arxiv_search(q):
    ARXIV_NAMESPACE = '{http://www.w3.org/2005/Atom}'

    url = f'http://export.arxiv.org/api/query?search_query=all:{q}&start=0&max_results=1'
    res = requests.get(url)
    et_root = ET.fromstring(res.content)

    for entry in et_root.findall(f"{ARXIV_NAMESPACE}entry"):
        title = entry.find(f"{ARXIV_NAMESPACE}title").text.strip()
        summary = entry.find(f"{ARXIV_NAMESPACE}summary").text.strip()
    return json.dumps({"title" : title, "summary" : summary})


def calculate(what):
    return eval(what)

## Prompt
Prompting is quite crucial for ReAct agents. They generally work by few-shot prompting. Meaning, we need to provide a few examples of what is expected of the agent in the prompt itself. For this reason, we need to write a detailed prompt with a few examples. Below is the prompt for the task.

Nothing fancy here other than pro-level prompting. We have listed all the actions, explicitly mentioned, “run in a loop of Thought, Action, PAUSE, Observation” so that the agent can invoke actions and think of the perceived observations.

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia:
Returns a summary from searching Wikipedia

arxiv_search:
e.g. arxiv_search:
Returns a summary of research papers

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

## Test the Agent
It's time to test out our ReAct agent. Let's collate all the actions in a dictionary, create an instance of the agent, and invoke its run method to run the agent.

In [ ]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "arxiv_search": arxiv_search
}

agent = Agent(prompt, max_turns=3, known_actions=known_actions)


def run_tests(question):
    bot = ChatBot(question)  # run chatbot without agent
    print(bot.run_llm())

    print(f"\n----------")
    agent.run(question)  # run agent

### 1st Question -- Easy

In [ ]:
question = "What is the capital of Indonesia?"
run_tests(question)

The capital of Indonesia is Jakarta.

----------

 Iteration: 1 

Thought: I should look up Indonesia on Wikipedia to find out its capital.  
Action: wikipedia: Indonesia  
PAUSE


 -- running wikipedia Indonesia   --
Next prompt: Observation: <span class="searchmatch">Indonesia</span>, officially the Republic of <span class="searchmatch">Indonesia</span>, is a country in Southeast Asia and Oceania, between the Indian and Pacific oceans. Comprising over 17

 Iteration: 2 

Answer: The capital of Indonesia is Jakarta.




### 2nd Question -- Easy

In [ ]:
question = "what is the sum of 7 and 5?"
run_tests(question)

The sum of 7 and 5 is 12.

----------

 Iteration: 1 

Thought: I can calculate the sum of 7 and 5 directly.  
Action: calculate: 7 + 5  
PAUSE


 -- running calculate 7 + 5   --
Next prompt: Observation: 12

 Iteration: 2 

Answer: The sum of 7 and 5 is 12.




### 3rd Question -- Difficult

In [ ]:
question = "explain the contrastive learning paper"
run_tests(question)

Contrastive learning is a type of self-supervised learning technique that aims to learn representations by contrasting positive and negative pairs of data points. The key idea is to bring similar data points closer together in the representation space while pushing dissimilar points apart. This approach has gained significant attention in various domains, including computer vision, natural language processing, and audio processing.

While I don't have access to specific papers published after October 2023, I can provide a general overview of the typical structure and findings of contrastive learning papers based on the existing literature up to that date.

### Key Components of Contrastive Learning Papers

1. **Introduction**:
   - The paper usually starts with an introduction to the problem being addressed and the motivation for using contrastive learning. It often highlights the limitations of supervised learning and the potential of self-supervised methods.

2. **Methodology**:
   -

In [ ]:
question = "explain the lightrag paper"
run_tests(question)

The "LightrAG" paper refers to a research work that presents a novel approach to the problem of image generation and manipulation using a lightweight architecture. While I don't have access to the specific content of the paper, I can provide a general overview of what such a paper might cover based on common themes in the field of generative models and lightweight architectures.

### Key Concepts Likely Covered in the LightrAG Paper:

1. **Generative Models**: The paper likely discusses generative models, which are algorithms that can generate new data instances that resemble a given dataset. Common examples include Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs).

2. **Lightweight Architecture**: The term "lightweight" suggests that the authors have designed a model that is efficient in terms of computational resources. This could involve reducing the number of parameters, optimizing the architecture for faster inference, or using techniques like pruning and

### 4th Qestion -- Difficult

In [ ]:
question = "What is AI Bites?"
run_tests(question)

AI Bites is a term that can refer to short, digestible pieces of information or insights related to artificial intelligence. This could include quick facts, tips, news updates, or summaries of AI concepts and technologies. The idea is to provide easily understandable and concise content that can help individuals stay informed about the rapidly evolving field of AI. If you have a specific context or topic in mind regarding AI Bites, feel free to share!

----------

 Iteration: 1 

Thought: I should look up "AI Bites" on Wikipedia to find out what it is.  
Action: wikipedia: AI Bites  
PAUSE


 -- running wikipedia AI Bites   --
Next prompt: Observation: &quot;Retrieval Augmented Generation(RAG) — A quick and comprehensive introduction&quot;. <span class="searchmatch">ai</span>-<span class="searchmatch">bites</span>.net. Kiela Douwe, Ho Alan (Oct 13, 2023). &quot;Where did Retrieval Augmented

 Iteration: 2 

Answer: AI Bites appears to be a platform or resource that provides quick and c

### 5th Question -- Difficult

In [ ]:
# Default answer is wrong. The answer after Wikipedia is write.
question = "Who is Alexander Tuzikov?"
run_tests(question)

As of my last knowledge update in October 2023, there is no widely recognized public figure or notable individual named Alexander Tuzikov. It's possible that he may be a private individual or a person who has gained prominence after that date. If you have more context or specific details about him, I may be able to provide more relevant information.

----------

 Iteration: 1 

Thought: I should look up "Alexander Tuzikov" on Wikipedia to find out more about him.  
Action: wikipedia: Alexander Tuzikov  
PAUSE


 -- running wikipedia Alexander Tuzikov   --
Next prompt: Observation: Ryabova (Deputy Minister of Communications and Informatization), <span class="searchmatch">Alexander</span> <span class="searchmatch">Tuzikov</span> [ru] (General Director of the United Institute of Informatics Problems

 Iteration: 2 

Answer: Alexander Tuzikov is the General Director of the United Institute of Informatics Problems.




In [ ]:
# Here the default answer is incorrect,
#   but after checking it on Wikipedia it turns out to be correct.
question = "Кто такой Александр Васильевич Тузиков?"
run_tests(question)

На момент моего последнего обновления информации в октябре 2023 года, нет широко известных данных о человеке по имени Александр Васильевич Тузиков. Возможно, это частное лицо или человек, не имеющий значительной публичной известности. Если у вас есть конкретные детали или контекст, связанные с этой персоной, я постараюсь помочь вам лучше.

----------

 Iteration: 1 

Thought: Я должен поискать информацию о "Александре Васильевиче Тузикове" в Википедии, чтобы узнать о нем больше.  
Action: wikipedia: Александр Васильевич Тузиков  
PAUSE


 -- running wikipedia Александр Васильевич Тузиков   --
Next prompt: Observation: <span class="searchmatch">Александр</span> <span class="searchmatch">Васильевич</span> <span class="searchmatch">Тузиков</span> (род. 5 сентября 1958 года, Полоцк) — белорусский учёный, c 2008 по 2022 год -генеральный директор Объединенного института

 Iteration: 2 

Answer: Александр Васильевич Тузиков (род. 5 сентября 1958 года, Полоцк) — белорусский учёный, который с 2

In [ ]:
# Wrong answer
question = "What are the scientific works of Alexandra Kosareva?"
run_tests(question)

As of my last update in October 2023, I do not have specific information on the scientific works of Alexandra Kosareva. To find her publications or contributions to scientific research, I recommend checking academic databases such as Google Scholar, PubMed, or institutional repositories. You can also look for her profile on research networking sites like ResearchGate or LinkedIn, where researchers often list their publications and projects. If you have a specific field or topic in mind, I can help guide you on how to search for relevant works.

----------

 Iteration: 1 

Thought: I should search for scientific works by Alexandra Kosareva to provide a summary of her contributions.  
Action: arxiv_search: Alexandra Kosareva  
PAUSE


 -- running arxiv_search Alexandra Kosareva   --
Next prompt: Observation: {"title": "All-optical attoclock for imaging tunnelling wavepackets", "summary": "Recent experiments on measuring time-delays during tunnelling of cold atoms\nthrough an optically cr

In [ ]:
# Wrong answer
question = "What are the scientific works of Dzmitry Paulenka?"
run_tests(question)

As of my last update in October 2023, I do not have specific information on the scientific works of Dzmitry Paulenka. To find his publications or contributions to scientific research, I recommend checking academic databases such as Google Scholar, ResearchGate, or institutional repositories. You can also look for his work in relevant journals or conference proceedings in his field of expertise. If you have access to a university library, they may also provide resources to help you find his research.

----------

 Iteration: 1 

Thought: I should search for scientific works by Dzmitry Paulenka to provide a summary of his contributions.  
Action: arxiv_search: Dzmitry Paulenka  
PAUSE


 -- running arxiv_search Dzmitry Paulenka   --
Next prompt: Observation: {"title": "The Decoration Theorem for Mandelbrot and Multibrot Sets", "summary": "We prove the decoration theorem for the Mandelbrot set (and Multibrot sets)\nwhich says that when a \"little Mandelbrot set\" is removed from the Mande

In [ ]:
# The default answer here is more detailed and complete
#   than what you would get from searching Wikipedia.
question = "Что такое эффект зловещей долины?"
run_tests(question)

Эффект зловещей долины (или "uncanny valley" на английском) — это концепция, описывающая реакцию людей на роботов, анимацию или другие искусственные объекты, которые выглядят почти, но не совсем, как реальные люди. Когда внешний вид объекта становится очень близким к человеческому, но все же содержит некоторые неестественные черты, это может вызывать у людей чувство дискомфорта или отторжения.

Термин был введен японским робототехником Масахиро Мори в 1970-х годах. Он заметил, что по мере того, как роботы становятся более человечными, положительная реакция людей на них возрастает, но только до определенного момента. Когда робот или анимация становятся почти, но не совсем человеческими, реакция начинает резко ухудшаться, создавая "долину" в графике, где уровень симпатии падает.

Эффект зловещей долины имеет важные последствия для дизайна роботов, анимации и виртуальной реальности, поскольку создатели стремятся создать более естественные и привлекательные образы, избегая при этом этого д